# I. Introduction

Saya Nurul Izzah dari hacktiv8 batch 017, saya membuat laporan ini bertujuan untuk menyelesaikan Milestone1. ini adalah dataset milik toko roti di kota Prancis. Dataset ini menyediakan detail transaksi harian pelanggan dari 2021-01-01 hingga 2022-09-30.

# II. Problem Identification

Banyak sekali makanan-makanan yang terkenal di negara perancis, salah satunya adalah bakery. data ini berisi tentang french daily bakery sales untuk mengetahui tentang :

- Produk apa dengan penjualan tertinggi?
- Di jam2 berapa produk banyak terjual?
- Di bulan apa terjadinya penurunan minat pembelian?
- Kapan terjadinya peningkatan minat pembeli bakery?
- Bagaimana tingkat pembelian produk berdasarkan bulan di tahun 2021-2022?

saya akan mencantumkan visualisasi data, statistik deskriptif, dan statistik inferensial dari problem statment yg telah dibuat.

# III. Data Loading

In [ ]:
!pip install google-cloud-bigquery # installasi untuk connect ke google bigquery
# menghubungkan ke big query
from google.cloud import bigquery

In [ ]:
#login dan mengautentikasi akun bigquery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

project_id = "dataset-371710"
client = bigquery.Client(project=project_id)

In [ ]:
client.query('''
SELECT *
FROM `dataset-371710.bakery_sales.french_bakery_sales` # melihat dataset distribution_centers
''').to_dataframe()

# Data Cleaning

In [ ]:
#import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
%matplotlib inline

In [ ]:
# melihat data
df = client.query('''
SELECT *
FROM `dataset-371710.bakery_sales.french_bakery_sales`
''').to_dataframe()

In [ ]:
df

In [ ]:
df.drop('int64_field_0', axis=1,inplace=True) # menhapus kolom yg tidak digunakan

In [ ]:
df['product_list'].value_counts() # meligar jumlah value di kolom product_list

In [ ]:
df # membuka data

In [ ]:
df.info() # melihat informasi ti[e data]

In [ ]:
df.loc[df['Quantity']<0] # melihat data yg minus

In [ ]:
len(df.loc[df['unit_price']<0]) # melihat data yg minus

In [ ]:
df['Quantity'] = df['Quantity'].abs() # mengembalikan nilai absolut

In [ ]:
df.loc[df['Quantity']<0] # melihat data kurang dari 0

In [ ]:
df['ticket_number'] = df['ticket_number'].astype(int) # mengganti tipe data

In [ ]:
df['Quantity'] = df['Quantity'].astype(int) # mengganti tipe data

In [ ]:
df # melihat data

In [ ]:
df['product_list'].unique() # melihat data yg unik/ nama yg tidak sama

In [ ]:
len(df.loc[df['product_list']=='.']) # melihat data .

In [ ]:
df.drop(df[df.product_list =='.'].index, inplace=True) # menghapus data .

# V. Analisis dan Perhitungan

In [ ]:
df.head() # melihat 5 data teratas

In [ ]:
df['price'] = df['Quantity'] * df['unit_price'] # mengkalikan data

In [ ]:
df.head() # melihat 5 data teratas

In [ ]:
len(df.loc[df['product_list']=='THE']) # melihat data THE

In [ ]:
df.describe() # melihat statistik daskriptif data

#Produk dengan penjualan tertinggi

In [ ]:
fig = plt.figure(figsize=(20, 12))

df['product_list'].value_counts(sort = True, ascending = False).head(10).plot(kind = 'bar',y='Product List',x='Sales', fontsize=18)
plt.grid(True)
fig.suptitle('PRODUCT TOTAL SALE', fontsize=20)
plt.ylabel('Sales', fontsize =18)
plt.xlabel('Product List', fontsize =18)

Dari data diatas dapat disimpulkan produk dengan penjualan tertinggi adalah Traditional Baguette.




#Di jam-jam berapa produk banyak terjual?


In [ ]:
df_hour = df.copy() # mencopy dataset

In [ ]:
df_hour.head() # melihat data set yg telah di copy

In [ ]:
df['date'].agg(['min','max']) # mengaggregasi

In [ ]:
df_hour['hour'] = pd.to_datetime(df['timehours'], format='%H:%M').dt.round('H').dt.hour

In [ ]:
df_hour

In [ ]:
df_hour.to_csv('clear_data2.csv', index=False)

In [ ]:
fig = plt.figure(figsize=(50, 12))

df_hour['hour'].value_counts(sort = True, ascending = False).head(10).plot(kind = 'pie',autopct='%.1f%%',fontsize=18)
plt.grid(True)
fig.suptitle('ARTICLE TOTAL SALE per HOUR', fontsize=20)
plt.xlabel('Hour', fontsize =18)

dari ata di atas, dapat disimpulkan produk terjual paling banyak terjadi di jam 9,10,11,12

#Bagaimana tingkat pembelian produk berdasarkan bulan di tahun 2021-2022?

In [ ]:
df_month = df.copy() # mengcopy dataset

In [ ]:
df_month['month'] = df['date'].astype(str).str[0:7] # mengganti tipe data

In [ ]:
df_month.drop(['ticket_number','unit_price'], axis=1, inplace=True) # menghapus kolom

In [ ]:
sales_per_month = df_month.groupby(['month']).sum() # menggabungkan data dan dijumlah

In [ ]:
sales_per_month # melihat dataset

In [ ]:
sales_per_month.plot(kind='line',figsize=(15, 10))
plt.grid(True)
plt.xlabel('Year-Month', fontsize =18)

dari data diatas dapat disimpulkan bahkan di penjualan meningkat pada bulan september 2021 dan 2022

In [ ]:
df # melihat dataset

#Cek outlier menggunakan boxplot

In [ ]:
df_tb = df.loc[df['product_list'] == 'TRADITIONAL BAGUETTE'] # untuk mengakses data

In [ ]:
df_tb # melihat data set

In [ ]:
figure=df_tb.boxplot(column='Quantity') # untuk membuat plot

dari visualisasi di atas, ada banyak outlier

In [ ]:
IQR= df['Quantity'].quantile(0.75) - df['Quantity'].quantile(0.25)
IQR #mengukur Rentang interkuartil untuk mengukur batas-batas outlier

In [ ]:
lower_bridge= df_tb['Quantity'].quantile(0.25)-(IQR*1.5)  #mengukur Rentang interkuartil untuk mengukur batas-batas outlier
upper_bridge= df_tb['Quantity'].quantile(0.75)+(IQR*1.5)
print(lower_bridge), print(upper_bridge) 

In [ ]:
df_tb['Quantity'].describe() # menghitung statistik deskriptif

In [ ]:
df_tb[df_tb['Quantity'] > upper_bridge] # mengelompokan data

In [ ]:
n_outlier = len(df_tb[df_tb['Quantity'] > upper_bridge]) # mengelompokan data

In [ ]:
mean_df_tb = df_tb['Quantity'].mean() # menghitung nilai rata-rata

In [ ]:
mean_df_tb # menampilkan nilai rata-rata

In [ ]:
df_tb.loc[df_tb['Quantity']>upper_bridge,'Quantity'] = mean_df_tb #  untuk mengakses single data

In [ ]:
df_tb # menampilkan data

In [ ]:
figure=df_tb.boxplot(column='Quantity') # membuat plot

In [ ]:
(n_outlier/len(df_tb))*100 # persentasi outlier

In [ ]:
df_tb.describe() # melihat statistik deskriptif

memilih mencari outlier menggunakan product list traditional baguagette karena merupakan produk yg paling banyak dibeli

#Measure of Variance

In [ ]:
df_tb['Quantity'].mean() # menghitung nilai rata-rata

In [ ]:
df_tb['Quantity'].var() # menghitung nilai varian

In [ ]:
df_tb['Quantity'].std() # menghitung standar deviasi

In [ ]:
df_tb['Quantity'].max() # menampilkan nilai maksimal

In [ ]:
df_tb['Quantity'].min() # menampilkan nilai minimal

- Nilai rata-ratanya berdasarkan Quantity = 1.5361

- Nilai variannya berdasarkan Quantity = 0.4214
Varian adalah ukuran dispersi data yang menunjukkan seberapa jauh nilai-nilai dalam sekumpulan data tersebar dari nilai rata-ratanya

- Nilai deviasainya berdasarkan Quantity = 0.6491
Standar deviasi adalah dispersi data yang menunjukkan seberapa jauh nilai-nilai dalam sekumpulan data tersebar dari nilai rata-ratanya

- Nilai maksimalnya berdasarkan Quantity = 3.0

- Nilai minimalnya berdasarkan Quantity = 1.0

#kapan terjadinya peningkatan minat pembeli bakery?

In [ ]:
sales_per_month.max()

In [ ]:
sales_per_month

pembelian paling banyak tejadi di bulan agustus 2021

#Resampling

In [ ]:
product_list = df.copy()

In [ ]:
product_list

In [ ]:
product_list.drop(['ticket_number'], axis=1, inplace=True)

In [ ]:
#product_list.drop(['price'], axis=1, inplace=True)

In [ ]:
product_list

In [ ]:
sns.displot(product_list.price.sample(6000), kde=True, label='Product sales per quantity')

In [ ]:
fig,ax = plt.subplots(ncols=4, figsize = (18,4))
for i in range(4):
  product_list.price.sample(400).plot(kind='hist',bins=20,ax=ax[i])
  ax[i].set_title('Sample #{}'.format(i+1))

Berdasarkan data yg telah di resampling dengan menggunakan jumlah sampling yg besar dan menggunakan jumlah sampling yg lebih kecil yg berjumlah 200 sebnayak 4x dapat dilihat bahwa disturbusi datanya mirip di mana distribusi data harga/price terbanyak berada di range 0 hingga 4. Dapat dikatakan bahwa bentuk distribusi data harga/price adalah positive long tail skewed

Link dashboard : https://datastudio.google.com/reporting/a28a7bcf-5d9a-4de5-a628-4d91d727c262

#VI. Pengambilan Kesimpulan

Dalam analisis kali ini saya menggunakan 5W1H yaitu 

- Produk apa dengan penjualan tertinggi?
produk dengan penjualan tertinggi adalah traditional baguette

- Di jam2 berapa produk banyak terjual?
sesuai dengan data yg dibuat di dashboard dan notebook, banyak pembelian di jam siang antara jam 10,11, dan 12 siang

- Di bulan apa terjadinya penurunan minat pembelian?
penurunan terjadi di bulan januari 2022

- Kapan terjadinya peningkatan minat pembeli bakery?
terjadi peningkatan penjualan pada bulan agustus 2021

- Bagaimana tingkat pembelian produk berdasarkan bulan di tahun 2021-2022?
dari analis di notebook, dapat disimpulkan bahkan di penjualan meningkat pada bulan september 2021 dan 2022

setelah menganalisis bisa disimpulkan diperlukan promosi agar menaikan insight perusahaan karena perusahaan banyak mengalami penurunan.